參考來源:https://blog.jiatool.com/posts/shopee_spider/

In [1]:
import requests
import urllib
import pandas as pd
import time
import random

In [2]:
#爬蟲 爬取蝦皮分類https://shopee.tw/**-cat.{categoryid}
def shopee_crawler(categoryid, newest):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.68',
        'x-api-source': 'pc',
        'referer': f'https://shopee.tw/search'
    }

    s = requests.Session()
    url = 'https://shopee.tw/api/v4/search/product_labels'
    r = s.get(url, headers=headers)
    
    '''
    categoryid 分類id 
    limit=60 每頁60個商品
    newest分頁 
    newest = 0 : 1~60 
             60 : 61~120
    '''
    url = f'https://shopee.tw/api/v4/search/search_items?by=relevancy&categoryids={categoryid}&limit=60&newest={newest}&order=desc&page_type=search&scenario=PAGE_OTHERS&version=2'
    r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        data = r.json()
    return data    

#id自動補零
def auto_addzero(num):
    if num<10:
        return '000' + str(num)
    elif num<100:
        return '00' + str(num)
    elif num<1000:
        return '0' + str(num)
    else:
        return str(num)
    
def avg_price(min_price, max_price):
    return int((min_price + max_price) / 200000)
    

In [3]:
class DfList: 

    def __init__(self):
        self.item_ids = []  
        self.names = []
        self.images = []
        self.prices = []
        self.categorys = []
        self.sort_by_ranks = []  
        self.sales_volumes = []
        self.zip_list = []
        
    def add_item_id(self, category, rank):
        itemid = category + auto_addzero(rank)
        self.item_ids.append(itemid)
        
    def add_name(self, name):
        self.names.append(name)
    
    def add_image(self, image):
        self.images.append(image)
        
    def add_price(self, min_price, max_price):
        price = avg_price(min_price, max_price)
        self.prices.append(price)
    
    def add_category(self, category):
        self.categorys.append(category)
    
    def add_sort_by_rank(self, sort_by_rank):
        self.sort_by_ranks.append(sort_by_rank)
        
    def add_sales_volume(self, sales_volume):
        self.sales_volumes.append(sales_volume)
        
    def get_zip_list(self):
        self.zip_list = [self.item_ids, self.names, self.images, self.prices, self.categorys, self.sort_by_ranks, self.sales_volumes]
        self.zip_list = list((zip(*self.zip_list)))
        return self.zip_list

In [4]:
columns=['item_id', 'name', 'image', 'price', 'category', 'sort_by_rank', 'sales_volume']
df = pd.DataFrame(columns = columns)
categorys = [['girls_clothes',11040766], 
            ['book',11041120],
            ['boys_clothes',11040730],
            ['home_life',11040925],
            ['fitness',11040840],
            ['food_souvenirs',11040483],
            ['shoes',11041795],
            ['auto_parts',11041308],
            ['girls_accessories_gold',11041224],
            ['game',11040812],
            ['beauty_care',11041691],
            ['entertainment_collection',11041645],
            ['baby_and_mother', 11040542],
            ['pet',11040647],
            ['girls_bags_boutique',11041292],
            ['mobile_tablet_and_peripherals',11041546],
            ['boys_bags_and_accessories',11041163],
            ['laptop',11041390],
            ['outdoors_travel',11042412],
            ['home_appliances_audio_and_video',11047569],
            ['cultural_and_creative_products',11040765],
            ['service_ticket',11041957],
            ['other',11041784]]


In [5]:
# i種分類
for i in range(len(categorys)):
    dflist = DfList()
    rank = 0
    
    #爬取100頁 但API裡的item有些會少幾個
    for j in range(120):
        categoryid = categorys[i][1]
        category = categorys[i][0]
        newest = 60 * j
        data = shopee_crawler(categoryid, newest)
        
        #隨機停幾秒
        time.sleep(random.uniform(2, 4))
        
        #每頁有60個商品 爬到6000時停止
        for k in range(len(data['items'])):
            rank += 1
            
            dflist.add_item_id(category, rank)
            dflist.add_name(data['items'][k]['item_basic']['name'])
            dflist.add_image(data['items'][k]['item_basic']['image'])
            dflist.add_price(data['items'][k]['item_basic']['price_min'], data['items'][k]['item_basic']['price_max'])
            dflist.add_category(category)
            dflist.add_sort_by_rank(rank)
            dflist.add_sales_volume(0)
            if rank == 6000:
                break
        if rank == 6000:
            break
    df = df.append(pd.DataFrame(dflist.get_zip_list(), columns = columns), ignore_index=True) 

In [6]:
df

,item_id,name,image,price,category,sort_by_rank,sales_volume
0,girls_clothes0001,朵莉團購🌝隨貨附發票 天天出現貨 三排 2扣/3扣/4扣 內衣 延長扣 台灣 加長扣 胸罩延...,9c7d978a75543c9bd30b271e145d5b31,2,girls_clothes,1,0
1,girls_clothes0002,台灣現貨 A027 海棉胸墊 內衣胸墊 泳衣胸墊 比基尼胸墊 上薄下厚胸墊 集中胸墊 增厚...,cb40c72c7288c7755f8e26e125910468,29,girls_clothes,2,0
2,girls_clothes0003,JS SHOP 台灣天天出貨【鋼圈集中款-雪紡罩衫三件組】A3021 男泳褲 韓國情侶泳裝 ...,e3befd3e4545bb6cb11fdb1efa4c9a78,354,girls_clothes,3,0
3,girls_clothes0004,&布料共和國&~竹節棉麻布~透氣棉麻縐布料.天然.吸濕透氣純色民族服裝布,c28c40fe9465e440e9689548811519cc,47,girls_clothes,4,0
4,girls_clothes0005,衝浪繩正宗泰國進口蠶絲蠟線 不退色 可帶著洗澡手作diy材料☆水晶森林手創館☆,1b171686050e6fafee1bb666ba62d76c,30,girls_clothes,5,0
...,...,...,...,...,...,...,...
137995,other5996,［AV1～情趣代購］台灣現貨 微愛minilove龍延油男用噴劑 男士私密處噴劑 JJ噴劑1...,d86d80b554193684d05b9ecc3064146d,99,other,5996,0
137996,other5997,(正港買一送一) 傳說中的 穿雲劍 穿梭自如 兩洞都可享用 一次兩種體驗 優客仕youcup...,0c22394c254f2d64e749cb95aeceb548,275,other,5997,0
137997,other5998,贈潤滑液 最新升級100頻矛盾大對決拓也哥推薦 超強按摩棒vs冷感絕無高潮女曲線型AV按摩棒...,d98e3636a82761de058fdbba9e59c97d,410,other,5998,0
137998,other5999,日本cosplay 死庫水情趣內衣 貓咪套裝 簍空刺繡 開胸內衣套裝 可愛 萌 獸娘 爆乳 貓耳,dabab2616aa7ee600a4663c7712515af,188,other,5999,0


In [7]:
df.to_csv('product.csv',index=False)